  Need to include:
  - Short mathemtatical explanation of the diffusion processination of mixtures, closed form mixture when diffused and conditional inverse problems
  - Introduction of the different components of the package (SDE, Denoiser, Timer, Integrator) joint with how diffusion models work. Plots to illustrate the components.
  - Explain different choice for each component
  - Plots to illustrate the diffusion on the mixture example

# Foreword: Gaussian Mixtures

## Conditional Gaussian Mixture Models for Diffusion-Based Inverse Problems

This module implements Bayesian inference with Gaussian mixture models (GMMs) in the context of diffusion processes and linear inverse problems.

## Mathematical Background

### 1. Gaussian Mixture Model (GMM)

A GMM represents a probability distribution as a weighted sum of $K$ Gaussian components:

$$p(x)=\sum_{i=1}^Kw_i\mathcal{N}(x;\mu_i,\Sigma_i)$$

Where:
- $w_i\geq0$, $\sum_iw_i=1$ (mixture weights)
- $\mu_i\in\mathbb{R}^d$ (component means)
- $\Sigma_i\in\mathbb{R}^{d\times d}$ (component covariance matrices)


### 3. Closed-Form Solution for GMM + Diffusion

When the prior is a GMM, the diffused distribution remains a GMM:

$$p_t(x_t)=\sum_iw_i\mathcal{N}(x_t;\mu_i(t),\Sigma_i(t))$$

Where:
- $\mu_i(t)=\sqrt{\alpha(t)}\mu_i(0)$
- $\Sigma_i(t)=\alpha(t)\Sigma_i(0)+(1-\alpha(t))I$
- $w_i(t)=w_i(0)$ (weights unchanged)

### 4. Bayesian Posterior with Linear Measurements

Given measurement: $y=Ax+\varepsilon$, where $\varepsilon\sim\mathcal{N}(0,\sigma_y^2I)$

The posterior is also a GMM:

$$p(x|y)=\sum_i\bar{w}_i\mathcal{N}(x;\bar{\mu}_i,\bar{\Sigma})$$

Where:
- $\bar{\Sigma}=\left(I+\frac{1}{\sigma_y^2}A^TA\right)^{-1}$
- $\bar{\mu}_i=\bar{\Sigma}\left(\frac{1}{\sigma_y^2}A^Ty+\mu_i\right)$
- $\bar{w}_i\propto w_i\times p(y|\mu_i)$
- $p(y|\mu_i)=\mathcal{N}(y;A\mu_i,\sigma_y^2I+AA^T)$

This conjugacy property makes GMMs particularly useful for diffusion-based inverse problems, as the posterior can be computed analytically at each step.

##  Diffusion Models

Typically, diffusion models can be described by a stochastic differential equation (SDE) of the form:
$$dx(t)=f(x, t)dt + g(t)dW(t)$$
It corresponds to slowly adding noise to the signal $x(t)$ over time such that the noised signal at time $t$ can be written as:
$$p(x(t)|x(0))=\mathcal{N}(x(t);s(t)x(0), s(t)^2\sigma(t)^2\mathbf{I})$$
or 
$$x(t) = s(t)x(0) + s(t)\sigma(t)\varepsilon, \quad \varepsilon\sim\mathcal{N}(0,I)$$

$s(t)$ describes how the original data $x(0)$ is attenuated or amplified over time as noise is added while $\sigma(t)$ it controls how much noise has been injected into the system at that time step. Both are avaible in closed form given $f(x, t)$ and $g(t)$.

$$
\boxed{s(t) \;=\; \exp\!\left(\int_0^t f(\xi)\, d\xi\right),  \quad
\sigma(t) \;=\; \left(\int_0^t \frac{g(\xi)^2}{s(\xi)^2}\, d\xi \right)^{1/2}}.
$$

The previous equation describes a way to add noise to a signal. It is possible to define a time reversed process that describes the denoising of the signal by an backward SDE:

$$dx=[f(x,t)−g(t)^2\nabla_x\log p_t(x)]dt+g(t)d\bar{W}(t)$$

### Generative Models
In order to generate new samples $x_0$ from pure noise $x_T$, diffusion models leverage the mathematical description of the denoising process defined above. The python class `Denoiser` is used to define the diffusion process starting from noise $x_T$ and denoising until new data $x_0$ is generated. It leverages the class `Integrator` to perform the numerical integration of the backward SDE. Possible choices of `Integrator` are: `EulerIntegrator`, `HeunIntegrator`, `DPMpp2sIntegrator`, `DDIMIntegrator`.

Most `Integrator` defined in the litterature necessitate $f$ and $g$ or $s$ and $\sigma$ to be defined. These attributes are defined in a `DiffusionModel` class.

The time discretization used in the `Denoiser` is defined in the `Timer` class. Possible choices of `Timer` are: `LinearTimer` or `CosineTimer`.

We also provide a `CondDenoiser` class to sample conditionally on a measurement $y$ to generate samples $x_0 \sim p(x_0|y)$.










### 

# Timer 